In [1]:
import numpy as np 
import math 
import rospy 
from tf import transformations
from std_msgs.msg import Header, ColorRGBA
from visualization_msgs.msg import Marker, MarkerArray
import sys
sys.path.append("..")
from kinematics.collision_checker import PyFCL
# from kinematics.robot import RobotClass 
from kinematics.rviz import RvizClass
print("Done.")

Done.


### Instance 

In [2]:
file_name = "../urdf/ur5e/ur5e_onrobot.urdf"
base_offset = [0.18,0.,0.79]
pyfcl = PyFCL()
rviz  = RvizClass(file_name=file_name, base_offset=base_offset)



In [16]:
import numpy as np 
import sys
sys.path.append("..")
from kinematics.utils.util_ik import make_ik_input
from kinematics.utils.util_structure import *
from kinematics.structure import CHAIN
from kinematics.collision_checker import PyFCL

class RobotClass(object):
    def __init__(self, file_name, base_offset, ctrl_joint_num=7):
        super(RobotClass,self).__init__() 
        self.chain = CHAIN(file_name=file_name, base_offset=base_offset)
        self.chain.add_joi_to_robot()
        self.chain.add_link_to_robot()
        self.chain.fk_chain(1)
        self.ctrl_joint_num = ctrl_joint_num
        self.fclize = PyFCL() 

    def solve_ik(self,  target_name      = ['wrist_3_joint'],
                        target_position  = [[0,0,0]],
                        target_rotation  = [[-1.57, 0, -1.57]],
                        solve_position   = [1],
                        solve_rotation   = [1],
                        weight_position  = 1,
                        weight_rotation  = 1,
                        joi_ctrl_num= 6): 
                        
        ingredients = self.make_ik_input(target_name = target_name,
                                    target_pos  = target_position,
                                    target_rot  = target_rotation,
                                    solve_pos   = solve_position,
                                    solve_rot   = solve_rotation,
                                    weight_pos  = weight_position,
                                    weight_rot  = weight_rotation,
                                    joi_ctrl_num= joi_ctrl_num)
        total_q = self.chain.get_q_from_ik(ingredients)
        ctrl_q  = total_q[:joi_ctrl_num+1] #Including base joint for Unity Env
        ctrl_q  = ctrl_q.reshape(-1,)
        return ctrl_q 

    def make_ik_input(self, target_name=["base_joint"], 
                    target_pos=[[0, 0, 0]], 
                    target_rot=[[0, 0, 0]], 
                    solve_pos=[1], 
                    solve_rot=[1], 
                    weight_pos=1, 
                    weight_rot=0, 
                    disabled_joi_id=[], 
                    joi_ctrl_num=7):
        return {"target_joint_name":target_name, 
                "target_joint_position":target_pos,
                "target_joint_rotation":target_rot,
                "solve_position":solve_pos,
                "solve_rotation":solve_rot,
                "position_weight":weight_pos,
                "rotation_weight":weight_rot,
                "disabled_joi_id":disabled_joi_id, 
                "joint_ctrl_num":joi_ctrl_num}
    
    def is_collision(self, q_list, objs=None):
        update_q_chain(self.chain.joint, q_list, self.ctrl_joint_num)
        self.chain.fk_chain(1)
        cap_R_list = get_cap_R_chain(self.chain.link)
        rpy_list = get_rpy_from_R_mat(cap_R_list)
        link_p_list = get_center_p_chain(self.chain.link)
        name_list = get_name_chain(self.chain.link)
        size_list = get_cap_size_chain(self.chain.link)
        scale_list = get_scale_chain(self.chain.link)                  
        height_list = get_height_chain(self.chain.link)
        radius_list = get_cap_radius(self.chain.link)
        for name, position, height, radius in zip(name_list, link_p_list, height_list, radius_list):
            print("name:{}, position:{}, height:{}, radius:{}".format(name, position, height, radius))
        # Capsulation 
        fcl_ingredients = get_cc_ingredients(name_list, link_p_list, rpy_list, height_list, radius_list)
        fclized_obj_lst = []
        for ingred in fcl_ingredients: 
            fclized_obj = self.fclize.fclize_obj(ingred)
            fclized_obj_lst.append(fclized_obj)
        print("total",len(fclized_obj_lst))

        # for idx, ingred in enumerate(fcl_ingredients):
        #     if idx == len(fcl_ingredients)-1:
        #         break 
        #     fclized_obj = fclized_obj_lst[idx]
        #     col = self.fclize.one2many_cc(fclized_obj, fclized_obj_lst[idx+2:])
        #     print("Is collision?", col)

        for idx1, ingred in enumerate(fcl_ingredients):
            if idx1 >8:
                break 
            print(fcl_ingredients[idx1]["name"])
            if fcl_ingredients[idx1]["name"] =="mount_link":
                pass 
            else: 
                fclized_obj1 = fclized_obj_lst[idx1]
                for idx2, fclized_obj2 in enumerate(fclized_obj_lst):
                    if idx2 >8:
                        break 
                    if fcl_ingredients[idx2]["name"] =="mount_link":
                        pass 
                    else: 
                        if idx1==idx2 or idx2==idx1+1 or idx2 ==idx1-1: 
                            col = False  
                        else:
                            col = self.fclize.one2one_cc(fclized_obj1, fclized_obj2)
                        if col:
                            print("Is collision?", col, fcl_ingredients[idx1]["name"]+" collision with "+fcl_ingredients[idx2]["name"])

In [17]:
robot = RobotClass(file_name=file_name, base_offset=base_offset)

q_list = np.array([0,0,0,0,0,0,0])
robot.is_collision(q_list)


name:base_link, position:[-3.61465827e-05 -1.87598935e-04  3.86934401e-02], height:0.0957, radius:0.0165
name:shoulder_link, position:[0.18042832 0.0110038  0.9497    ], height:0.0155, radius:0.0891
name:upper_arm_link, position:[0.3985     0.12896669 0.95303687], height:0.412, radius:0.0944
name:forearm_link, position:[0.79797109 0.01855672 0.9530565 ], height:0.388, radius:0.0698
name:wrist_1_link, position:[0.99699697 0.1191     0.9515    ], height:0.0094, radius:0.0729
name:wrist_2_link, position:[0.99698031 0.1353     0.8559    ], height:0.0186, radius:0.0602
name:wrist_3_link, position:[0.99708857 0.2054     0.85238258], height:0.0026, radius:0.0443
name:mount_link, position:[0.997 0.179 0.853], height:0.0001, radius:0.0001
name:camera_base_link, position:[0.99700006 0.179      0.946     ], height:0.0001, radius:0.0001
name:gripper_base_link, position:[0.99695416 0.29179972 0.85262635], height:0.0784, radius:0.0384
name:gripper_tcp_link, position:[0.99608264 0.44140814 0.86313484